In [1]:
from foundations.projectables import SympyProjectable
from foundations.projectables import ProjectableModel,ProjectableIntersection
from foundations.projectables import restructure
from foundations.projectables import merge_with_coupling
from foundations.projectables import strategy_eliminate_feedfwd
from modeling.compute import create_vars, Par, Var
from foundations.functional_noobj import eliminate_vars
from foundations.functional_noobj import concatenate_residuals
from foundations.functional_noobj import optimizer_solver
from foundations.functionals import encode_sympy
from modeling.execution import sympy_to_edges
from graph.graphutils import default_tree
from presolver.tearprepare import execute_tearing
from graph.operators import reformulate
from numpy import pi

ModuleNotFoundError: No module named 'foundations'

# Disciplines

### Hydro

In [2]:
Df, Ds, Dd, tf, ts, td, hf, mtot = create_vars('Df Ds Dd tf ts td hf m_{platform}')
g, rhow = 9.81, 1023.6

Hydro = ProjectableModel()
Vd = Hydro.Var('Vd', pi/4*(Df**2*hf+Ds**2*ts+Dd**2*td))
FB = Hydro.Var('F_B', rhow*Vd*g)
FW = Hydro.Var('F_W', FB)
# center of buoyancy above the keel calculations
xd,xs = td/2,td+ts/2
xf1,xf2 = td+ts+hf/2, td+ts+tf/2
totA1, totA2 = hf*Df+ts*Ds+td*Dd, tf*Df+ts*Ds+td*Dd
KB = Hydro.Var('K_B', (hf*Df*xf1+ts*Ds*xs+td*Dd*xd)/totA1) 
KG = Hydro.Var('K_G', (tf*Df*xf2+ts*Ds*xs+td*Dd*xd)/totA2) 
I = Hydro.Var('I', pi/64*Df**4)
BM = Hydro.Var('B_M', I/Vd)
#should be 3-5% according to http://web.mit.edu/13.012/www/handouts/Reading3.pdf
GM = Hydro.Var('G_M', KB+BM-KG) 
C33 = Hydro.Var('C_{33}', pi*rhow*g/4*Df**2)
A33 = Hydro.Var('A_{33}', 0.0525*pi*rhow*(Dd**3+Ds**3+Df**3))
omega0 = Hydro.Var('\omega_0', (C33/(A33+mtot))**1/2)

In [7]:
yH = {Df:2, Ds:2, Dd:2, tf:0.1, ts:0.1, td:0.1, hf:0.9*0.3, mtot:800}
HF = Hydro.functional()
HF.dict_in_dict_out(yH, cleanup=True), '{:.2f} %'.format(HF.dict_in_dict_out(yH, cleanup=True)[BM]*100)

({Vd: 1.4765485471872024,
  F_B: 14826.785861357048,
  F_W: 14826.785861357048,
  K_B: 0.23500000000000007,
  K_G: 0.15,
  I: 0.785398163397448,
  B_M: 0.5319148936170212,
  G_M: 0.6169148936170212,
  C_{33}: 31546.35289650436,
  A_{33}: 4051.825142670288,
  \omega_0: 3.2509779277765007},
 '53.19 %')

### Mass

In [8]:
mbatt, A_s = create_vars('m_{batt} A_s')
mprop, mcomms, eta_solar, rho, rhoh = 20, 20, 10, 700, 2700
Mass = ProjectableModel()
Mass.add_equation(mtot, FW/g)
msolar = Mass.Var('m_{solar}', eta_solar*A_s)
mstruct = Mass.Var('m_{struct}', mtot-mbatt-msolar-mcomms-mprop) 
Mass.add_equation(td, (4/pi*mstruct-Df**2*tf*rho-Ds**2*ts*rho)/(Dd**2*rhoh))

In [9]:
yM = {mbatt:100, A_s:0.5, FW:36130}
MF = Mass.functional()
MF.dict_in_dict_out({**yH, **yM}, cleanup=True)

{m_{platform}: 3682.976554536202,
 m_{solar}: 5.0,
 m_{struct}: 3537.976554536202,
 td: 0.3652492275538281}

### Combined

In [10]:
combined_projectables = Hydro.projectables+Mass.projectables
HydroMass = ProjectableIntersection(*combined_projectables)
HydroMass.solvepar(x_initial={mtot:500, td:0.01})

In [12]:
y = {Df:2, Ds:2, Dd:2, tf:0.1, ts:0.1, hf:0.9, mbatt:0, A_s:0.5}
HMF = HydroMass.functional()
z = HMF.dict_in_dict_out(y, cleanup=True)
z

{m_{platform}: 4883.212631203285,
 td: 0.5185373748041826,
 Vd: 4.770625860886348,
 F_B: 47904.31591210404,
 F_W: 47904.31591210404,
 K_B: 0.7592686874020914,
 K_G: 0.35926868740209134,
 I: 0.785398163397448,
 B_M: 0.16463210201345083,
 G_M: 0.5646321020134508,
 C_{33}: 31546.35289650436,
 A_{33}: 4051.825142670288,
 \omega_0: 1.7653172652916602,
 m_{solar}: 5.0,
 m_{struct}: 4838.212631203285}

In [34]:
HMcoupling = merge_with_coupling(Hydro, Mass)
HMcoupling.solvepar(x_initial={mtot:500, td:0.01, FW:10000})
HMcouplingF = HydroMass.functional()
z = HMcouplingF.dict_in_dict_out(y, cleanup=True)

In [58]:
z

{m_{platform}: 4883.212631203285,
 td: 0.5185373748041826,
 Vd: 4.770625860886348,
 F_B: 47904.31591210404,
 F_W: 47904.31591210404,
 K_B: 0.7592686874020914,
 K_G: 0.35926868740209134,
 I: 0.785398163397448,
 B_M: 0.16463210201345083,
 G_M: 0.5646321020134508,
 C_{33}: 31546.35289650436,
 A_{33}: 4051.825142670288,
 \omega_0: 1.7653172652916602,
 m_{solar}: 5.0,
 m_{struct}: 4838.212631203285}

In [36]:
HMR = HMcoupling.residuals()

In [37]:
HMcoupling.required_solvevars()

(F_W, m_{platform}, td)

In [38]:
from jax import jacfwd
g = jacfwd(HMR.flat_in_flat_out)

In [47]:
from modeling.arghandling import decode, flatten_args, unflatten_args, EncodedFunction
import numpy as np
def adjoints(projectable):
    H = projectable.residuals()
    F = projectable.functional()
    g = jacfwd(H.flat_in_flat_out)
    N = sum(map(sum, F.decoder.shapes))
    def calculate(*args):
        x_F =  F.encoder.decode(args)
        x0 = {**x_F, **F.dict_out_only(*args)}
        x0_np = H.encoder.encode(x0, flatten=True)
        d = decode(flatten_args(g(x0_np).T), 
                   H.encoder.order, 
                   [(N,) for elt in H.encoder.order], unflatten=True)
        grad_h_y = np.vstack(F.decoder.encode(d)).T
        grad_h_x = np.vstack(F.encoder.encode(d)).T
        inv_grad_h_y = np.linalg.inv(grad_h_y)
        DJ = -np.dot(inv_grad_h_y, grad_h_x)
        return unflatten_args(flatten_args(DJ), [(N,) for elt in F.encoder.order])
    return EncodedFunction(calculate, F.encoder)

In [48]:
HMcouplingA = adjoints(HMcoupling)

In [57]:
# import cProfile
# import re
# cProfile.run('HMcouplingA.dict_in_flat_out(y)')

In [56]:
HMcouplingA.dict_in_flat_out(y)

DeviceArray([ 3.76358302e+03,  3.76358302e+04,  5.08083708e+04,
              4.44102797e+04,  7.27595761e-12, -1.31725406e+04,
             -5.98992842e+01, -5.98992842e+00,  3.83647607e+02,
              3.83647607e+03,  5.17924269e+03,  4.52704176e+03,
              9.09494702e-13, -1.34276662e+03, -6.10594130e+00,
             -6.10594130e-01,  1.93032689e-02,  1.93032689e-01,
              6.10594130e-01,  5.07778573e-01, -5.18537375e-01,
             -4.17561441e-01, -1.89877050e-03, -1.89877050e-04],            dtype=float64)

In [39]:
x0 = HMR.encoder.encode({**z, **y}, flatten=True)

In [40]:
HMR.flat_in_flat_out(x0)

DeviceArray([0.00000000e+00, 0.00000000e+00, 1.11022302e-16],            dtype=float64, weak_type=True)

In [45]:
g(x0)

DeviceArray([[ 1.00000000e+00,  0.00000000e+00, -3.15463529e+04,
              -3.15463529e+03, -3.15463529e+04, -3.15463529e+04,
              -2.83917176e+04, -1.63579630e+04,  0.00000000e+00,
               0.00000000e+00,  0.00000000e+00],
             [-1.01936799e-01,  1.00000000e+00,  0.00000000e+00,
               0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
               0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
               0.00000000e+00,  0.00000000e+00],
             [-1.20175892e-05,  0.00000000e+00,  1.00000000e+00,
               2.59259259e-02,  2.59259259e-01,  0.00000000e+00,
               2.59259259e-02,  5.18537375e-01,  2.59259259e-01,
               1.17892550e-03,  1.17892550e-04]],            dtype=float64, weak_type=True)

# Restructuring

In [8]:
merged_eqs = [(eq.right, eq.var) for eq in combined_projectables]

In [9]:
edges = sympy_to_edges(merged_eqs, tvar=None,
                       filterto=lambda x: not x.always_input)

In [10]:
not_input = [omega0, A33, BM, KG, I, mstruct, KB, msolar]
not_output = [Df, Dd, Ds, ts, tf, td, mbatt, A_s]

In [11]:
xout = execute_tearing(edges, not_input=not_input, not_output=not_output)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-07


In [12]:
outset_initial = {key:val[0] for key,val in edges[1].items()}
_, new_tree = reformulate(edges, default_tree(edges[0].keys()), edges[1], xout)

In [13]:
new_eqs = restructure(merged_eqs, xout)

In [14]:
new_eqs_order = [new_eqs[idx] for idx in new_tree[0].keys()]

### Restructured system

In [15]:
RS = ProjectableIntersection(*(SympyProjectable(right, var)
                        for var, right in new_eqs_order))

In [16]:
RSF = RS.functional()

In [17]:
RSF.encoder.order[3] == list(y.keys())[3]

False

In [18]:
y = {Df:2, Ds:2, Dd:2, tf:0.1, ts:0.1, td:0.1, mbatt:0, A_s:0.5}
RSF.dict_in_dict_out(y)

{I: 0.785398163397448,
 C_{33}: 31546.35289650436,
 A_{33}: 4051.825142670288,
 K_G: 0.15,
 m_{struct}: 1288.0529879718201,
 m_{solar}: 5.0,
 m_{platform}: 1333.0529879718201,
 \omega_0: 2.9291612670854152,
 F_W: 13077.249812003556,
 F_B: 13077.249812003556,
 Vd: 1.3023182766430443,
 hf: 0.21454078241331698,
 B_M: 0.6030769724141095,
 K_B: 0.20727039120665852,
 G_M: 0.6603473636207681}

In [19]:
#F,R,solvervars = feedback_residuals(new_eqs_order)
#Rsys = concatenate_residuals((Hydro.residuals(), 
#                             Mass.residuals()))

# Optimization

### Multidisciplinary Feasible with original equations

In [22]:
RSF.encoder.order

(Df, Ds, Dd, td, tf, ts, A_s, m_{batt})

In [23]:
x0 = {Df:2, Ds:1.5, Dd:2, tf:0.15, ts:5, td:0.2, A_s:0.1, mbatt:100}
#x0 = z
x0_all = {**RSF.dict_in_dict_out(x0), **x0} # Initial guesses for all variables

In [24]:
obj  = encode_sympy(mtot)
ineq = concatenate_residuals((
    #encode_sympy(-hf),
    encode_sympy(hf-0.9*tf),
    encode_sympy(Ds-0.9*Df),
    encode_sympy(Ds-0.9*Dd),
    encode_sympy(GM-0.05),
    encode_sympy(0.03-GM),
    encode_sympy(0.1-td)))

In [25]:
HydroMass.solvepar(x_initial={mtot:100, td:0.1})
HMF = HydroMass.functional()
obj_mdf, ineq_mdf = (eliminate_vars(obj, HMF), 
                    eliminate_vars(ineq, HMF))

In [26]:
OPT_MDF = optimizer_solver(obj_mdf, ineqs=(ineq_mdf,), bounds={
    Df:(.1,10), Ds:(.1,10), Dd:(.1,10), 
    tf:(.1,10), ts:(.1,.5), td:(.1,10), A_s:(.1,10), mbatt:(.1,1000)})

In [27]:
x0_mdf = {key:val for (key,val) in x0_all.items() if key in OPT_MDF.decoder.order}
x0_mdf[Df] = x0_mdf[Df]

In [28]:
obj_mdf.dict_in_only(x0_mdf)

DeviceArray([8352.33779832], dtype=float64)

In [136]:
z1 = OPT_MDF.dict_in_dict_out(x0=x0_mdf)

Optimization terminated successfully


### Multidisciplinary Feasible with restructured equations

In [29]:
x0 = {Df:2, Ds:1.5, Dd:2, tf:0.15, ts:5, td:0.2, A_s:0.1, mbatt:100}

In [30]:
eliminate_vars(obj, RSF).dict_in_flat_out(x0)#, eliminate_vars(ineq, RSF).dict_in_flat_out(x0)

DeviceArray([8352.33779832], dtype=float64, weak_type=True)

In [31]:
obj_el, ineq_el = (eliminate_vars(obj, RSF), 
                    eliminate_vars(ineq, RSF))

In [32]:
OPT = optimizer_solver(obj_el, ineqs=(ineq_el,), bounds={
    Df:(.1,10), Ds:(.1,10), Dd:(.1,10), 
    tf:(.1,10), ts:(.1,.5), td:(.1,10), A_s:(.1,10), mbatt:(.1,1000)})

In [33]:
z = OPT.dict_in_dict_out(x0=x0)

Optimization terminated successfully


In [36]:
z, obj_el.dict_in_only(z), ineq_el.dict_in_only(z)

({Df: 0.7133636651119954,
  Ds: 0.1,
  Dd: 0.11111111101233184,
  td: 0.1,
  tf: 0.46881162270138726,
  ts: 0.5,
  A_s: 0.1,
  m_{batt}: 0.1},
 DeviceArray([177.62895723], dtype=float64),
 DeviceArray([ 1.22509003e-10, -5.42027299e-01,  8.89013446e-11,
              -3.92121908e-03, -1.60787809e-02,  0.00000000e+00],            dtype=float64))

### All At Once

In [36]:
R_AAO = HydroMass.residuals()

In [37]:
OPT_AAO = optimizer_solver(obj, ineqs=(ineq,), eqs=(R_AAO,), bounds={
    Df:(.1,10), Ds:(.1,10), Dd:(.1,10), 
    tf:(.1,10), ts:(.1,.5), td:(.1,10), A_s:(.1,10), mbatt:(.1,1000)})

In [38]:
x0 = {Df:1, Ds:1.5, Dd:0.1, tf:0.15, ts:5, td:0.2, A_s:0.1, mbatt:0.1}
x1 = {**RSF.dict_in_dict_out(x0), **x0}
x1 = {key:val for (key,val) in x1.items() if key in OPT_AAO.decoder.order}
obj.dict_in_flat_out(x1), ineq.dict_in_only(x1), R_AAO.dict_in_only(x1)

(array([6312.81849399]),
 array([-3.53458681,  0.6       ,  1.41      , -0.75845784,  0.73845784,
        -0.1       ]),
 array([ 1.59872116e-14,  7.27595761e-12,  0.00000000e+00,  4.44089210e-16,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.13686838e-13,  0.00000000e+00, -2.45563569e-11,
         0.00000000e+00,  0.00000000e+00, -2.01505479e-14]))

In [39]:
z1 = OPT_AAO.dict_in_dict_out(x0=x1)

Optimization terminated successfully


In [40]:
obj.dict_in_flat_out(z1)

array([177.62895769])

### Individual Discipline Feasible

In [41]:
F, R, solvevars = HydroMass.functional_parts()
Relim = eliminate_vars(R, F)

In [42]:
obj_idf_el, ineq_idf_el, eq_idf_el = (eliminate_vars(obj, F), 
                          eliminate_vars(ineq, F),
                          eliminate_vars(R, F))

In [43]:
OPT_IDF = optimizer_solver(obj_idf_el, ineqs=(ineq_idf_el,), eqs=(eq_idf_el,), bounds={
    Df:(.1,10), Ds:(.1,10), Dd:(.1,10), 
    tf:(.1,10), ts:(.1,.5), td:(.1,10), A_s:(.1,10), mbatt:(.1,1000)})

In [44]:
x0 = {Df:1, Ds:1.5, Dd:0.1, tf:0.15, ts:5, td:0.2, A_s:0.1, mbatt:0.1}
x2 = {**RSF.dict_in_dict_out(x0), **x0}
x2 = {key:val for (key,val) in x2.items() if key in OPT_IDF.decoder.order}
(eliminate_vars(obj, F).dict_in_flat_out(x2), 
eliminate_vars(ineq, F).dict_in_flat_out(x2),
eliminate_vars(R, F).dict_in_flat_out(x2))

(array([6312.81849399]),
 array([-3.53458681,  0.6       ,  1.41      , -0.75845784,  0.73845784,
        -0.1       ]),
 array([-7.27595761e-12, -2.01505479e-14]))

In [45]:
z2 = OPT_IDF.dict_in_dict_out(x0=x2)

Optimization terminated successfully


In [46]:
obj.dict_in_flat_out(z2)

array([177.62895833])